# Implementación Básica de K-means en 3D

En este notebook se implementa el algoritmo **k-means** de forma muy básica y sin usar librerías especializadas para el clustering. El proceso consiste en:
- Leer y transformar un archivo CSV.
- Reemplazar valores vacíos en la columna *Estado* con el valor más frecuente.
- Mapear los valores de *Tipo de contagio* y *Estado* a números.
- Aplicar k-means sobre 3 variables: *Edad*, *Tipo de contagio* y *Estado*.
- Mostrar los resultados.

El archivo a usar se encuentra en la ruta: `/workspaces/ml-infotep/G1_Miercoles/hw3/G7/datoscovid.zip`


In [ ]:
# FUNCION: Leer y transformar datos del CSV
def leer_y_transformar_tres(archivo):
    """
    Esta función lee un archivo CSV (con ; como separador) y extrae 3 variables:
      - Edad (posición 7)
      - Tipo de contagio (posición 10)
      - Estado (posición 12)

    Se convierte Edad a float y se mapean las variables categóricas a números.
    Además, se reemplazan los valores vacíos en Estado con el valor más frecuente (modo).

    Retorna:
      - datos: lista de tuplas con 3 valores (edad, tipo, estado)
      - mapa_tipo: diccionario con el mapeo de Tipo de contagio
      - mapa_estado: diccionario con el mapeo de Estado
    """
    # Definir los índices según la posición en el archivo
    idx_edad = 7
    idx_tipo = 10
    idx_estado = 12

    # Leer todas las líneas del archivo
    with open(archivo, "r", encoding="latin1") as f:
        lineas = f.read().splitlines()

    # Primera pasada: calcular el valor más frecuente en 'Estado'
    freq_estado = {}
    for linea in lineas[1:]:
        campos = linea.split(";")
        if len(campos) <= idx_estado:
            continue
        estado_val = campos[idx_estado].strip()
        if estado_val == "":
            continue  # ignorar vacíos para el conteo
        if estado_val in freq_estado:
            freq_estado[estado_val] += 1
        else:
            freq_estado[estado_val] = 1

    # Determinar el modo (valor más frecuente)
    modo = None
    max_count = -1
    for key, count in freq_estado.items():
        if count > max_count:
            max_count = count
            modo = key
    # Si no se encontró ninguno, se asigna "Leve" por defecto
    if modo is None:
        modo = "Leve"

    # Segunda pasada: procesar cada línea y reemplazar los vacíos en Estado
    datos = []
    mapeo_tipo = {}
    mapeo_estado = {}
    codigo_tipo = 0
    codigo_estado = 0

    for linea in lineas[1:]:
        campos = linea.split(";")
        if len(campos) <= idx_estado:
            continue
        try:
            edad = float(campos[idx_edad])
        except:
            continue  # Si no se puede convertir Edad, se omite la fila

        # Extraer variables categóricas y limpiar espacios
        tipo = campos[idx_tipo].strip()
        estado = campos[idx_estado].strip()
        if estado == "":
            estado = modo  # Reemplazar valor vacío con el modo

        # Asignar un número a cada valor de 'Tipo de contagio'
        if tipo not in mapeo_tipo:
            mapeo_tipo[tipo] = codigo_tipo
            codigo_tipo += 1
        # Asignar un número a cada valor de 'Estado'
        if estado not in mapeo_estado:
            mapeo_estado[estado] = codigo_estado
            codigo_estado += 1

        # Crear la tupla con 3 valores
        punto = (edad, mapeo_tipo[tipo], mapeo_estado[estado])
        datos.append(punto)
    return datos, mapeo_tipo, mapeo_estado

# Probar la función de lectura usando la ruta fija del archivo
archivo = "/workspaces/ml-infotep/G1_Miercoles/hw3/G7/datoscovid.csv"
datos, mapa_tipo, mapa_estado = leer_y_transformar_tres(archivo)
print("Se han cargado", len(datos), "registros válidos.")
print("Mapeo de Tipo de contagio:", mapa_tipo)
print("Mapeo de Estado:", mapa_estado)


## Explicación de la Función de Lectura y Transformación

En esta sección se:
- Lee el archivo CSV usando la ruta `/workspaces/ml-infotep/G1_Miercoles/hw3/G7/datoscovid.csv`.
- Se cuentan los valores en la columna *Estado* (ignorando los vacíos) para encontrar el valor más frecuente.
- Se reemplazan los valores vacíos en *Estado* por este valor.
- Se convierten las variables categóricas (*Tipo de contagio* y *Estado*) en números.
- Finalmente, se crea una lista de puntos, cada uno representado por una tupla (Edad, número de Tipo de contagio, número de Estado).


In [ ]:
# FUNCION: Algoritmo k-means básico en 3D
def kmeans_3d(datos, k, umbral=0.001, max_iter=100):
    """
    Aplica el algoritmo k-means a la lista 'datos' (cada dato es una tupla de 3 dimensiones).
    Retorna:
      - centroides: lista de centroides finales (tuplas)
      - clusters: lista de clusters (cada cluster es una lista de puntos)
    """
    n = len(datos)
    if n == 0 or k <= 0:
        print("No hay datos o k no es válido.")
        return [], []

    # Función interna para calcular la distancia Euclidiana en 3D
    def distancia(p1, p2):
        return ((p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 + (p1[2]-p2[2])**2)**0.5

    # Generador pseudoaleatorio para elegir centroides iniciales
    def pseudo_random(seed):
        a = 1103515245
        c = 12345
        m = 2**31
        while True:
            seed = (a * seed + c) % m
            yield seed / m

    pseudo = pseudo_random(1)
    centroides = []
    indices_usados = []
    # Elegir k centroides iniciales de forma aleatoria
    for i in range(k):
        rand_val = next(pseudo)
        idx = int(rand_val * n)
        while idx in indices_usados:
            rand_val = next(pseudo)
            idx = int(rand_val * n)
        indices_usados.append(idx)
        centroides.append(datos[idx])

    # Iterar para ajustar los centroides
    for iteracion in range(max_iter):
        # Paso 1: Asignar cada punto al centroide más cercano
        clusters = [[] for _ in range(k)]
        for punto in datos:
            distancias = [distancia(punto, centro) for centro in centroides]
            indice_min = distancias.index(min(distancias))
            clusters[indice_min].append(punto)

        # Paso 2: Recalcular los centroides como promedio de cada cluster
        nuevos_centroides = []
        for i in range(k):
            if len(clusters[i]) == 0:
                nuevos_centroides.append(centroides[i])
            else:
                suma = [0.0, 0.0, 0.0]
                for p in clusters[i]:
                    suma[0] += p[0]
                    suma[1] += p[1]
                    suma[2] += p[2]
                promedio = (suma[0] / len(clusters[i]),
                            suma[1] / len(clusters[i]),
                            suma[2] / len(clusters[i]))
                nuevos_centroides.append(promedio)

        # Calcular el cambio total en los centroides
        cambio_total = sum(distancia(centroides[i], nuevos_centroides[i]) for i in range(k))
        if cambio_total < umbral:
            print("Convergencia alcanzada en la iteración", iteracion + 1)
            return nuevos_centroides, clusters
        centroides = nuevos_centroides

    print("Se alcanzó el máximo de iteraciones.")
    return centroides, clusters

# Probar el algoritmo con k clusters (pediremos el valor al usuario)
k = int(input("Ingresa el número de clusters (k): "))
centroides_finales, clusters_finales = kmeans_3d(datos, k)

print("\nCentroides finales:")
for i, centro in enumerate(centroides_finales):
    print("Cluster", i, ":", centro)
for i, cluster in enumerate(clusters_finales):
    print("Cluster", i, "tiene", len(cluster), "puntos.")
